# Zgradite regresijski model: linearni in polinomski regresijski modeli


## Linearna in polinomska regresija za določanje cen buč - Lekcija 3
<p>
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika: Dasani Madipalli</figcaption>


<!--![Infografika: Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->

#### Uvod

Do sedaj ste raziskali, kaj regresija je, z vzorčnimi podatki, zbranimi iz nabora podatkov o cenah buč, ki ga bomo uporabljali skozi celotno lekcijo. Prav tako ste ga vizualizirali z uporabo `ggplot2`. 💪

Zdaj ste pripravljeni, da se poglobite v regresijo za strojno učenje. V tej lekciji boste izvedeli več o dveh vrstah regresije: *osnovni linearni regresiji* in *polinomski regresiji*, skupaj z nekaj matematike, ki stoji za temi tehnikami.

> V tem učnem načrtu predpostavljamo minimalno matematično predznanje in si prizadevamo, da bi bila vsebina dostopna študentom iz drugih področij. Zato bodite pozorni na opombe, 🧮 poudarke, diagrame in druga učna orodja, ki vam bodo pomagala pri razumevanju.

#### Priprava

Naj vas spomnimo, da nalagate te podatke, da bi si zastavili vprašanja o njih.

-   Kdaj je najboljši čas za nakup buč?

-   Kakšno ceno lahko pričakujem za zabojček miniaturnih buč?

-   Ali naj jih kupim v polovičnih košarah ali v škatlah velikosti 1 1/9 busha? Poglobimo se v te podatke.

V prejšnji lekciji ste ustvarili `tibble` (sodobno reinterpretacijo podatkovnega okvira) in ga napolnili z delom izvirnega nabora podatkov, pri čemer ste standardizirali cene glede na bushel. S tem ste sicer pridobili približno 400 podatkovnih točk, vendar le za jesenske mesece. Morda lahko pridobimo še malo več podrobnosti o naravi podatkov, če jih še bolj očistimo? Bomo videli... 🕵️‍♀️

Za to nalogo bomo potrebovali naslednje pakete:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [zbirka paketov za R](https://www.tidyverse.org/packages), zasnovana za hitrejše, enostavnejše in zabavnejše delo z znanostjo o podatkih!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) je [okvir paketov](https://www.tidymodels.org/packages/) za modeliranje in strojno učenje.

-   `janitor`: Paket [janitor](https://github.com/sfirke/janitor) ponuja preprosta orodja za pregledovanje in čiščenje "umazanih" podatkov.

-   `corrplot`: Paket [corrplot](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) omogoča vizualno raziskovanje korelacijske matrike, ki podpira samodejno preurejanje spremenljivk za odkrivanje skritih vzorcev med njimi.

Pakete lahko namestite z naslednjim ukazom:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Spodnji skript preveri, ali imate nameščene potrebne pakete za dokončanje tega modula, in jih po potrebi namesti.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

## 1. Linearna regresijska premica

Kot ste se naučili v Lekciji 1, je cilj linearne regresije narisati *premico* *najboljše prileganje*, da:

-   **Prikažete odnose med spremenljivkami**. Prikažete odnos med spremenljivkami.

-   **Napovedujete**. Naredite natančne napovedi, kje bi nov podatek padel v odnosu do te premice.

Za risanje takšne premice uporabljamo statistično tehniko, imenovano **Regresija najmanjših kvadratov**. Izraz `najmanjši kvadrati` pomeni, da so vsi podatkovni točki okoli regresijske premice kvadrirani in nato sešteveni. Idealno je, da je končni seštevek čim manjši, saj želimo nizko število napak oziroma `najmanjše kvadrate`. Tako je premica najboljše prileganje tista, ki nam daje najnižjo vrednost za vsoto kvadriranih napak - od tod ime *regresija najmanjših kvadratov*.

To počnemo, ker želimo modelirati premico, ki ima najmanjšo kumulativno razdaljo od vseh naših podatkovnih točk. Prav tako kvadriramo izraze pred seštevanjem, saj nas zanima njihova velikost in ne smer.

> **🧮 Pokaži mi matematiko**
>
> Ta premica, imenovana *premica najboljše prileganje*, se lahko izrazi z [enačbo](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` je '`pojasnjevalna spremenljivka` ali `napovednik`'. `Y` je '`odvisna spremenljivka` ali `rezultat`'. Naklon premice je `b`, `a` pa je presečišče z osjo y, kar se nanaša na vrednost `Y`, ko `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "naklon = $y/x$")
    Infografika: Jen Looper
>
> Najprej izračunajte naklon `b`.
>
> Z drugimi besedami, in glede na prvotno vprašanje o podatkih o bučah: "napovedati ceno buče na koš po mesecih", bi `X` pomenil ceno, `Y` pa mesec prodaje.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Infografika: Jen Looper
> 
> Izračunajte vrednost Y. Če plačujete približno 4 USD, mora biti april!
>
> Matematika, ki izračuna premico, mora prikazati naklon premice, ki je odvisen tudi od presečišča, oziroma kje se `Y` nahaja, ko `X = 0`.
>
> Metodo izračuna teh vrednosti si lahko ogledate na spletni strani [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Obiščite tudi [ta kalkulator najmanjših kvadratov](https://www.mathsisfun.com/data/least-squares-calculator.html), da vidite, kako vrednosti številk vplivajo na premico.

Ni tako strašno, kajne? 🤓

#### Korelacija

Še en izraz, ki ga je treba razumeti, je **Koeficient korelacije** med danima spremenljivkama X in Y. S pomočjo razsevnega diagrama lahko hitro vizualizirate ta koeficient. Diagram s podatkovnimi točkami, razporejenimi v urejeni premici, ima visoko korelacijo, medtem ko diagram s podatkovnimi točkami, razpršenimi povsod med X in Y, ima nizko korelacijo.

Dober model linearne regresije bo tisti, ki ima visok (bližje 1 kot 0) koeficient korelacije z uporabo metode regresije najmanjših kvadratov s premico regresije.


## **2. Ples s podatki: ustvarjanje podatkovnega okvira za modeliranje**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Umetniško delo @allison_horst</figcaption>


<!--![Umetniško delo \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Naložite potrebne knjižnice in podatkovni niz. Podatke pretvorite v podatkovni okvir, ki vsebuje podmnožico podatkov:

-   Uporabite samo buče, katerih cena je določena na sod.

-   Datum pretvorite v mesec.

-   Izračunajte ceno kot povprečje najvišje in najnižje cene.

-   Ceno prilagodite tako, da odraža določanje cen glede na količino v sodu.

> Te korake smo obravnavali v [prejšnji lekciji](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

V duhu čiste avanture raziščimo [`paket janitor`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor), ki ponuja preproste funkcije za pregledovanje in čiščenje neurejenih podatkov. Na primer, poglejmo imena stolpcev za naše podatke:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Lahko naredimo bolje. Spremenimo ta imena stolpcev v `friendR` tako, da jih pretvorimo v konvencijo [snake_case](https://en.wikipedia.org/wiki/Snake_case) z uporabo `janitor::clean_names`. Če želite izvedeti več o tej funkciji: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Veliko urejenosti 🧹! Zdaj pa ples s podatki z uporabo `dplyr`, kot v prejšnji lekciji! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Odlično delo!👌 Zdaj imate čist in urejen nabor podatkov, na katerem lahko zgradite svoj novi regresijski model!

Kaj pravite na razpršeni diagram?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Raztreseni diagram nas spomni, da imamo podatke le za mesece od avgusta do decembra. Verjetno potrebujemo več podatkov, da bi lahko sklepali na linearen način.

Poglejmo si še enkrat naše podatke za modeliranje:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Kaj če bi želeli napovedati `ceno` buče na podlagi stolpcev `mesto` ali `paket`, ki sta tipa znak? Ali pa še bolj preprosto, kako bi lahko našli korelacijo (ki zahteva, da sta oba njena vnosa številska) med, recimo, `paketom` in `ceno`? 🤷🤷

Modeli strojnega učenja najbolje delujejo s številski lastnostmi namesto z besedilnimi vrednostmi, zato je običajno potrebno pretvoriti kategorijske lastnosti v številske predstavitve.

To pomeni, da moramo najti način za preoblikovanje naših napovednih spremenljivk, da jih model lahko učinkoviteje uporabi, kar imenujemo `inženiring lastnosti`.


## 3. Predobdelava podatkov za modeliranje z recepti 👩‍🍳👨‍🍳

Dejavnosti, ki preoblikujejo vrednosti napovedovalcev, da jih model lažje učinkovito uporabi, se imenujejo `inženiring značilnosti`.

Različni modeli imajo različne zahteve glede predobdelave. Na primer, metoda najmanjših kvadratov zahteva `kodiranje kategornih spremenljivk`, kot so mesec, sorta in ime_mesta. To preprosto pomeni `pretvorbo` stolpca s `kategorničnimi vrednostmi` v enega ali več `številskih stolpcev`, ki nadomestijo izvirni stolpec.

Na primer, predpostavimo, da vaši podatki vključujejo naslednjo kategornično značilnost:

|   mesto   |
|:---------:|
|  Denver   |
|  Nairobi  |
|   Tokio   |

Uporabite lahko *ordinalno kodiranje*, da vsaki kategoriji dodelite edinstveno celoštevilsko vrednost, kot je to:

| mesto |
|:-----:|
|   0   |
|   1   |
|   2   |

In to bomo storili s svojimi podatki!

V tem razdelku bomo raziskali še en izjemen paket Tidymodels: [recipes](https://tidymodels.github.io/recipes/) - ki je zasnovan za pomoč pri predobdelavi podatkov **preden** treniramo model. V svojem jedru je recept objekt, ki določa, katere korake je treba uporabiti na podatkovnem naboru, da ga pripravimo za modeliranje.

Zdaj pa ustvarimo recept, ki pripravi naše podatke za modeliranje tako, da za vse opazovanja v stolpcih napovedovalcev nadomesti edinstveno celoštevilsko vrednost:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Super! 👏 Pravkar smo ustvarili naš prvi recept, ki določa izid (ceno) in ustrezne napovedne spremenljivke ter da morajo biti vsi stolpci napovednih spremenljivk kodirani v nabor celih števil 🙌! Hitro si poglejmo podrobnosti:

-   Klic funkcije `recipe()` s formulo pove receptu *vloge* spremenljivk, pri čemer uporablja podatke `new_pumpkins` kot referenco. Na primer, stolpec `price` je bil dodeljen vlogi `outcome` (izid), medtem ko so bili preostali stolpci dodeljeni vlogi `predictor` (napovednik).

-   `step_integer(all_predictors(), zero_based = TRUE)` določa, da morajo biti vsi napovedniki pretvorjeni v nabor celih števil, pri čemer se številčenje začne pri 0.

Prepričani smo, da imate misli, kot so: "To je tako kul!! Ampak kaj, če bi moral potrditi, da recepti dejansko delajo točno to, kar pričakujem? 🤔"

To je odlična misel! Vidite, ko je vaš recept definiran, lahko ocenite parametre, potrebne za dejansko predobdelavo podatkov, in nato pridobite obdelane podatke. Tega običajno ne potrebujete, ko uporabljate Tidymodels (v trenutku bomo videli običajno prakso -> `workflows`), vendar je to lahko koristno, ko želite narediti nekakšen pregled za potrditev, da recepti delajo, kar pričakujete.

Za to boste potrebovali še dva glagola: `prep()` in `bake()`, in kot vedno, vam naši mali R prijatelji, ki jih je ustvarila [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations), pomagajo bolje razumeti to temo!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Umetniško delo @allison_horst</figcaption>


<!--![Umetniško delo \@allison_horst](../../../../../../2-Regression/3-Linear/images/recipes.png){width="550"}-->


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): oceni potrebne parametre iz učnega nabora, ki jih je mogoče kasneje uporabiti na drugih podatkovnih naborih. Na primer, za določen stolpec napovednika, kateremu opazovanju bo dodeljena cela števila 0, 1, 2 itd.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): vzame pripravljeni recept in uporabi operacije na katerem koli podatkovnem naboru.

Torej, pripravimo in uporabimo naše recepte, da resnično potrdimo, da bodo stolpci napovednikov v ozadju najprej kodirani, preden se model prilega.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Woo-hoo!🥳 Obdelani podatki `baked_pumpkins` imajo vse napovedne spremenljivke kodirane, kar potrjuje, da bodo koraki predobdelave, opredeljeni kot naš recept, delovali po pričakovanjih. To sicer otežuje branje, a je veliko bolj razumljivo za Tidymodels! Vzemite si čas, da ugotovite, katera opazovanja so bila preslikana v ustrezne celoštevilske vrednosti.

Prav tako je vredno omeniti, da je `baked_pumpkins` podatkovni okvir, na katerem lahko izvajamo izračune.

Na primer, poskusimo najti dobro korelacijo med dvema točkama vaših podatkov, da bi potencialno zgradili dober napovedni model. Za to bomo uporabili funkcijo `cor()`. Vnesite `?cor()`, da izveste več o funkciji.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Kot se izkaže, obstaja le šibka povezava med mestom in ceno. Vendar pa je nekoliko boljša povezava med paketom in njegovo ceno. To ima smisel, kajne? Običajno velja, da večja kot je škatla s pridelki, višja je cena.

Medtem pa poskusimo vizualizirati tudi korelacijsko matriko vseh stolpcev z uporabo paketa `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Veliko bolje.

Dobro vprašanje, ki si ga lahko zdaj zastavimo glede teh podatkov, je: '`Kakšno ceno lahko pričakujem za določen paket buč?`' Pojdimo kar takoj k stvari!

> Opomba: Ko **`bake()`** uporabite na pripravljenem receptu **`pumpkins_prep`** z **`new_data = NULL`**, pridobite obdelane (tj. kodirane) podatke za učenje. Če bi imeli drug nabor podatkov, na primer testni nabor, in bi želeli videti, kako bi recept predhodno obdelal te podatke, bi preprosto uporabili **`bake`** na **`pumpkins_prep`** z **`new_data = test_set`**.

## 4. Izdelava modela linearne regresije

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika avtorja Dasani Madipalli</figcaption>


<!--![Infografika avtorja Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Zdaj, ko smo sestavili recept in dejansko potrdili, da bodo podatki ustrezno predobdelani, bomo zgradili regresijski model, da odgovorimo na vprašanje: `Kakšno ceno lahko pričakujem za določen paket buč?`

#### Učenje linearnega regresijskega modela z uporabo učnega nabora

Kot ste verjetno že ugotovili, je stolpec *price* `izhodna` spremenljivka, medtem ko je stolpec *package* `napovedna` spremenljivka.

Za to bomo najprej razdelili podatke tako, da bo 80 % šlo v učni nabor in 20 % v testni nabor, nato pa definirali recept, ki bo kodiral napovedni stolpec v niz celih števil, in zgradili specifikacijo modela. Recepta ne bomo pripravili in uporabili, saj že vemo, da bo podatke predobdelal, kot je pričakovano.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Odlično! Zdaj, ko imamo recept in specifikacijo modela, moramo najti način, kako ju združiti v objekt, ki bo najprej predobdelal podatke (v ozadju prep+bake), nato prilegal model na predobdelane podatke in omogočal tudi morebitne aktivnosti po obdelavi. Kako ti je to všeč za mirno vest!🤩

V Tidymodels se ta priročen objekt imenuje [`workflow`](https://workflows.tidymodels.org/) in priročno združuje tvoje komponente modeliranja! To je tisto, kar bi v *Pythonu* imenovali *pipelines*.

Torej, združimo vse skupaj v workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

Poleg tega je mogoče potek dela prilagoditi/usposobiti na zelo podoben način, kot se lahko model.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Iz modelnega izhoda lahko vidimo koeficiente, pridobljene med učenjem. Ti predstavljajo koeficiente premice najboljše prileganja, ki nam daje najnižjo skupno napako med dejansko in napovedano spremenljivko.

#### Ocenjevanje uspešnosti modela z uporabo testnega nabora

Čas je, da preverimo, kako se je model odrezal 📏! Kako to storimo?

Zdaj, ko smo model naučili, ga lahko uporabimo za napovedovanje na testnem_naboru z uporabo `parsnip::predict()`. Nato lahko te napovedi primerjamo z dejanskimi vrednostmi oznak, da ocenimo, kako dobro (ali ne!) model deluje.

Začnimo z izdelavo napovedi za testni nabor in nato združimo stolpce s testnim naborom.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Da, pravkar ste trenirali model in ga uporabili za napovedovanje! 🔮 Je dober? Poglejmo, kako dobro deluje model!

V Tidymodels to naredimo z uporabo `yardstick::metrics()`! Pri linearni regresiji se osredotočimo na naslednje metrike:

-   `Root Mean Square Error (RMSE)`: Kvadratni koren [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). To daje absolutno metriko v isti enoti kot oznaka (v tem primeru cena buče). Manjša kot je vrednost, boljši je model (v preprostem smislu predstavlja povprečno ceno, za katero so napovedi napačne!).

-   `Coefficient of Determination (običajno znan kot R-squared ali R2)`: Relativna metrika, pri kateri višja vrednost pomeni boljše prileganje modela. V bistvu ta metrika predstavlja, koliko variance med napovedanimi in dejanskimi vrednostmi oznak model lahko pojasni.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Tam gre uspešnost modela. Poglejmo, ali lahko dobimo boljšo indikacijo z vizualizacijo razpršenega grafa paketa in cene, nato pa uporabimo napovedi za dodajanje črte najboljše prileganje.

To pomeni, da bomo morali pripraviti in obdelati testni niz, da kodiramo stolpec paketa, nato pa to povežemo z napovedmi, ki jih je ustvaril naš model.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Odlično! Kot lahko vidiš, linearni regresijski model ne posploši najbolje odnosa med paketom in njegovo ustrezno ceno.

🎃 Čestitke, pravkar si ustvaril model, ki lahko pomaga napovedati ceno nekaj vrst buč. Tvoja praznična bučna njiva bo čudovita. Ampak verjetno lahko ustvariš še boljši model!

## 5. Ustvari polinomski regresijski model

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika: Dasani Madipalli</figcaption>


<!--![Infografika: Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Včasih naši podatki morda nimajo linearne povezave, vendar še vedno želimo napovedati rezultat. Polinomska regresija nam lahko pomaga pri napovedovanju bolj zapletenih nelinearnih povezav.

Vzemimo na primer povezavo med embalažo in ceno v našem naboru podatkov o bučah. Čeprav je včasih med spremenljivkami linearna povezava - večja kot je buča po volumnu, višja je cena - te povezave včasih ni mogoče prikazati kot ravnino ali ravno črto.

> ✅ Tukaj so [nekateri dodatni primeri](https://online.stat.psu.edu/stat501/lesson/9/9.8) podatkov, ki bi lahko uporabili polinomsko regresijo
>
> Ponovno si oglejte povezavo med sorto in ceno na prejšnjem grafu. Ali se vam zdi, da bi moral ta raztros nujno analizirati z ravno črto? Morda ne. V tem primeru lahko poskusite polinomsko regresijo.
>
> ✅ Polinomi so matematični izrazi, ki lahko vsebujejo eno ali več spremenljivk in koeficientov

#### Učimo polinomski regresijski model z uporabo učnega nabora

Polinomska regresija ustvari *ukrivljeno črto*, ki bolje ustreza nelinearnim podatkom.

Poglejmo, ali bo polinomski model boljši pri napovedovanju. Sledili bomo nekoliko podobnemu postopku kot prej:

-   Ustvarite recept, ki določa korake predobdelave, ki jih je treba izvesti na naših podatkih, da jih pripravimo za modeliranje, tj. kodiranje napovedovalcev in izračunavanje polinomov stopnje *n*

-   Zgradite specifikacijo modela

-   Združite recept in specifikacijo modela v delovni tok

-   Ustvarite model z ujemanjem delovnega toka

-   Ocenite, kako dobro model deluje na testnih podatkih

Pojdimo kar v akcijo!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Ocenjevanje zmogljivosti modela

👏👏 Ustvarili ste polinomski model, zdaj pa naredimo napovedi na testnem naboru!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Juhu, ocenimo, kako se je model odrezal na testnem naboru z uporabo `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Veliko boljša zmogljivost.

`rmse` se je zmanjšal s približno 7 na približno 3, kar kaže na zmanjšano napako med dejansko ceno in napovedano ceno. To lahko *približno* interpretiramo kot povprečno napako napovedi, ki znaša približno 3 USD. `rsq` se je povečal s približno 0,4 na 0,8.

Vse te metrike kažejo, da polinomski model deluje veliko bolje kot linearni model. Odlično delo!

Poglejmo, ali lahko to vizualiziramo!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Lahko vidite ukrivljeno črto, ki se bolje prilega vašim podatkom! 🤩

To lahko naredite še bolj gladko, če podate polinomsko formulo funkciji `geom_smooth`, kot to:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Tako kot gladka krivulja!🤩

Tukaj je, kako bi naredili novo napoved:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Napoved modela `polynomial` je smiselna, glede na razpršene grafe `price` in `package`! In, če je to boljši model kot prejšnji, ob pogledu na iste podatke, morate načrtovati proračun za te dražje buče!

🏆 Odlično opravljeno! Ustvarili ste dva regresijska modela v eni lekciji. V zadnjem delu o regresiji se boste naučili o logistični regresiji za določanje kategorij.

## **🚀Izziv**

Preizkusite več različnih spremenljivk v tej beležki, da vidite, kako korelacija vpliva na natančnost modela.

## [**Kvizi po predavanju**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Pregled in samostojno učenje**

V tej lekciji smo se naučili o linearni regresiji. Obstajajo tudi druge pomembne vrste regresije. Preberite o tehnikah Stepwise, Ridge, Lasso in Elasticnet. Dober tečaj za poglobljeno učenje je [Stanfordov tečaj statističnega učenja](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Če želite izvedeti več o uporabi izjemnega okvira Tidymodels, si oglejte naslednje vire:

-   Spletna stran Tidymodels: [Začnite z Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn in Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **POSEBNA ZAHVALA:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) za ustvarjanje izjemnih ilustracij, ki naredijo R bolj prijazen in privlačen. Več ilustracij najdete v njeni [galeriji](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da se zavedate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
